In [4]:
# Imports

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

from adquire_functions import get_students_and_katas
from cleaning_funcions import clean_katas
from chosing_functions import display_presenter_last_kata


# functions
def adquire(): 
    return get_students_and_katas()

def cleaning(df_students, df_katas): 
    return df_students, clean_katas(df_katas)

def get_completed_by_user(user, host='https://www.codewars.com/api/v1/users/', 
                          path='/code-challenges/completed'):
    url = host + user + path
    resp = requests.get(url)    
    return resp.json()

def user_time_kata(user, slug, completed, katas): 
    res = '2090-01-01T00:00:00.000Z'# default
    for data in completed[user]: 
        if data['slug'] == slug: 
            res = data['completedAt']
            break
    return res

def get_info_dict(df_students): 
    completed = {u: get_completed_by_user(u) for u in df_students.index}
    res_dict = dict( [(k, v['data']) for k, v in completed.items()] )
    return res_dict

def create_df_checking(df_students, df_katas): 
    katas = df_katas.index
    students = df_students.index
    
    # Creamos un DataFrame: Students x Katas
    df_res = pd.DataFrame(index=df_students.index, columns=df_katas.index)
    
    # tarda un rato
    completed_katas_dict = get_info_dict(df_students)
       
    # cada celda es el tiempo que ese student(row) tardó en realizar la kata(column)
    for user in students: 
        for slug in katas: 
            df_res.loc[user, slug] = user_time_kata(user, slug, completed_katas_dict, katas)
    
    # convertimos las columnas a tipo Datetime
    for col in df_res.columns: 
        df_res[col] = pd.to_datetime(df_res[col],infer_datetime_format=True)
        
    # En este punto df_res son valores temporales    
    # copiamos la solución en otro DF porque en el bucle la segunda vez da problemas al cambiar el tipo
    df_sol = df_res.copy()

    # display(df_res)
    for user in students: 
        df_sol.loc[user] = df_res.loc[user] < df_katas.limit
    
    return df_sol

def save_results(df, folder='./output/'): 
    df.to_csv(folder + 'output.csv', index=True) 
    
    writer = pd.ExcelWriter(folder + 'output.xlsx')
    df.to_excel(writer, 'Sheet1')
    # df2.to_excel(writer,'Sheet2')
    writer.save()
    
def color_negative_red(x):
    """
    Takes a boolean and returns a string with
    the css property `'color: red'` for False
    strings, green otherwise.
    """
    color = 'red' if x == False else 'green'
    return 'color: {}'.format(color) 

def display_with_colors(df): 
    df_style = df.style.applymap(color_negative_red)
    display(df_style)

    
# Main    
    
def main(): 
    df_students, df_katas = adquire()
    df_students, df_katas = cleaning(df_students, df_katas)
    df = create_df_checking(df_students, df_katas)
    
    # outputs
    display_presenter_last_kata(df)       
    display_with_colors(df)   
    save_results(df)


if __name__ == '__main__':
    main()